In [ ]:
!pip install monai torch pytorch-ignite pandas pyyaml munch imageio tqdm opencv-python nibabel "protobuf<=3.20.3" matplotlib matplotlib-inline


In [ ]:
!pip install sagemaker boto3 awscli

In [1]:
!aws s3 ls --profile asher

2025-06-19 16:34:55 sagemaker-ap-southeast-1-345594598345


In [2]:
local_dir = "./data"

In [1]:
import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker.local import LocalSession
role = get_execution_role() # "arn:aws:iam::345594598345:role/service-role/AmazonSageMaker-ExecutionRole-20250707T163996"
print("Role: ", role)
# local_sess = LocalSession()  # for local session
# local_sess.config = {'local': {'local_code': True}}

sess = sagemaker.Session(boto_session=boto3.Session(region_name="ap-southeast-1", profile_name="asher")) #name of sso session
# #for remote session
settings = dict(
    sagemaker_session=sess,
    role=role,
    instance_type="ml.t3.large",
    dependencies='./requirements.txt',
)

bucket_name = sess.default_bucket()
print(bucket_name)

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/asherlim/Library/Application Support/sagemaker/config.yaml


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│    2 import boto3                                                                                │
│    3 from sagemaker import get_execution_role                                                    │
│    4 from sagemaker.local import LocalSession                                                    │
│ ❱  5 role = get_execution_role() # "arn:aws:iam::345594598345:role/service-role/AmazonSageMak    │
│    6 print("Role: ", role)                                                                       │
│    7 # local_sess = LocalSession()  # for local session                                          │
│    8 # local_sess.config = {'local': {'local_code': True}}                                       │
│                                                                                                  │
│ /Users/asherlim/miniforge3/envs/p158/lib/python3.12/site-packages/sagemaker/session.py:8035 in   │
│ get_execution_role                                                                               │
│                                                                                                  │
│   8032 │   """                                                                                   │
│   8033 │   if not sagemaker_session:                                                             │
│   8034 │   │   sagemaker_session = Session()                                                     │
│ ❱ 8035 │   arn = sagemaker_session.get_caller_identity_arn()                                     │
│   8036 │                                                                                         │
│   8037 │   if ":role/" in arn:                                                                   │
│   8038 │   │   return arn                                                                        │
│                                                                                                  │
│ /Users/asherlim/miniforge3/envs/p158/lib/python3.12/site-packages/sagemaker/session.py:6103 in   │
│ get_caller_identity_arn                                                                          │
│                                                                                                  │
│   6100 │   │   │   "sts",                                                                        │
│   6101 │   │   │   region_name=self.boto_region_name,                                            │
│   6102 │   │   │   endpoint_url=sts_regional_endpoint(self.boto_region_name),                    │
│ ❱ 6103 │   │   ).get_caller_identity()["Arn"]                                                    │
│   6104 │   │                                                                                     │
│   6105 │   │   role = re.sub(r"^(.+)sts::(\d+):assumed-role/(.+?)/.*$", r"\1iam::\2:role/\3", a  │
│   6106                                                                                           │
│                                                                                                  │
│ /Users/asherlim/miniforge3/envs/p158/lib/python3.12/site-packages/botocore/client.py:601 in      │
│ _api_call                                                                                        │
│                                                                                                  │
│    598 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    599 │   │   │   │   )                                                                         │
│    600 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  601 │   │   │   return self._make_api_call(operation_nam

In [ ]:
# !aws s3 cp ./data s3://{bucket_name}/prostate158 --recursive --profile asher
# print("Done!")

In [4]:
if bucket_name:
    train_channel = f"s3://{bucket_name}/prostate158/prostate158_train/train"
    test_channel = f"s3://{bucket_name}/prostate158/prostate158_test/test"
inputs = {"train": train_channel, "test": test_channel}
print(inputs)

{'train': 's3://sagemaker-ap-southeast-1-345594598345/prostate158/prostate158_train/train', 'test': 's3://sagemaker-ap-southeast-1-345594598345/prostate158/prostate158_test/test'}


In [ ]:
%%time
from sagemaker.pytorch.estimator import PyTorch
import os


def run_training():
    estimator = PyTorch(
        entry_point="train_gpt.py",
        source_dir=".",
        role=role,
        region="ap-southeast-1",
        instance_type="ml.t3.large",
        instance_count=1,
        framework_version="2.0.0",
        enable_spot_training=True,  # for cost savings
        max_run=3600,
        max_wait=4*3600,
        input_mode="File",
        sagemaker_session=sess,
        inputs={
            "training": train_channel,
            "training_config": os.environ.get("SM_CHANNEL_TRAINING_CONFIG", "tumor.yaml"),
        }
    )
    estimator.fit(inputs)

run_training()

CPU times: user 1min 14s, sys: 2.43 s, total: 1min 17s
Wall time: 1min 18s
